# bike-reloot data set
## assembling a dat set for a prototype 

This notebook has the purpose of making a data set for the _bike-reloot_ project. 

Steps are the following: 
- scrape will haben bike pictures
- rename them to "stolen" and "not_stolen" with consecutive IDs

---

Inside of a loop through a list of willhaben URLs with different "&pages=1": 

Structure of the "a-tag" I want to scrap, which is one entry of a bike.

```
<a 
    href="/iad/kaufen-und-verkaufen/d/bike-discounter-angebot-kross-trans-4-0-s-28-zoll-0401-578744109/" 
    id="search-result-entry-header-578744109" 
    aria-labelledby="search-result-entry-header-578744109" 
    class="sc-ca51e2d8-0 lkgIJQ sc-deecb898-1 imWndm" 
    data-testid="search-result-entry-header-578744109">
</a>
```

Then retreive the href and save it to a dataframe, for n number of pages which lead to n number of entries (say, 600 entries).


In [1]:
import requests
from bs4 import BeautifulSoup

# DEFINE BASE URL 
# = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-4552&rows=1"
url_fahrräder = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-radsport/fahrraeder-4552"
# and &page=9 &rows=1

# MAKE REQUEST ON BASE URL 
page = requests.get(url_fahrräder)
soup = BeautifulSoup(page.content, "html.parser")

# Define a function to filter elements based on their IDs within the div with ID "skip-to-resultlist"
def find_by_id_custom(tag):
    return (
        tag.name == 'a' and 
        tag.get('id', '').startswith('search-result-entry-header') and 
        tag.has_attr('id')
    )

# FIND ALL a tags within ID
links = soup.find(id="skip-to-resultlist").find_all(find_by_id_custom)
print(len(links))
print(links[0].get('href'))



5
/iad/kaufen-und-verkaufen/d/geero-2-touring-classic-cream-inkl-abus-absperrkette-city-chain-pedelec-e-bike-769424965/


In [13]:
import pandas as pd

list_id = [ link.get('id') for link in links]
list_href = [ link.get('href') for link in links]

df = pd.DataFrame({"id": list_id,
                   "href": list_href})

# split ID into all parts and extract the unique ID 
df['id_int'] = df['id'].str.split('-',expand=True)[4]
df['links'] = "https://www.willhaben.at"+df['href']
df['links'][1]
df

,id,href,id_int,links
0,search-result-entry-header-769424965,/iad/kaufen-und-verkaufen/d/geero-2-touring-cl...,769424965,https://www.willhaben.at/iad/kaufen-und-verkau...
1,search-result-entry-header-769424561,/iad/kaufen-und-verkaufen/d/ridley-rennrad-cyc...,769424561,https://www.willhaben.at/iad/kaufen-und-verkau...
2,search-result-entry-header-769424327,/iad/kaufen-und-verkaufen/d/mountainbike-20-zo...,769424327,https://www.willhaben.at/iad/kaufen-und-verkau...
3,search-result-entry-header-769423720,/iad/kaufen-und-verkaufen/d/einkaufskorb-fuer-...,769423720,https://www.willhaben.at/iad/kaufen-und-verkau...
4,search-result-entry-header-769423545,/iad/kaufen-und-verkaufen/d/radon-slide-160mm-...,769423545,https://www.willhaben.at/iad/kaufen-und-verkau...


# Extracting URL links of images of bikes

In [3]:
import requests
from bs4 import BeautifulSoup
from PIL import Image

In [40]:
import requests
from bs4 import BeautifulSoup
import time

df['links'][1]


# TEST THE SINGLE ENTRY IMG EXTRACTION
single_page = requests.get(df['links'][1])
single_soup = BeautifulSoup(single_page.content, "html.parser")

def find_by_data_test_id_custom(tag):
    return (
        tag.name == 'img' and 
        tag.get('data-testid', '').startswith('image') and 
        tag.has_attr('alt')
    )

print(len(single_soup.find_all(find_by_data_test_id_custom)))
img_links = single_soup.find_all(find_by_data_test_id_custom)

# print(single_soup.find_all('img'))

print([ link.get('src') if link.has_attr('src') 
       else link.get('data-flickity-lazyload') 
       for link in img_links ])

print([ link.get('alt')  for link in img_links ])
#img_links



17
['https://cache.willhaben.at/mmo/1/769/424/561_1110958152.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_1794694154.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_1880484276.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_960294679.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_1295309011.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-356202943.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-84416267.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-681580211.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-1413034130.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-424420074.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-775431706.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-245896278.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-273355802.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-1358047413.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_-1607734713.jpg', 'https://cache.willhaben.at/mmo/1/769/424/561_2121

In [59]:

# PERFORM IMAGE EXTRACTION ON WHOLE DF
img_list = {}
for i in df.index:
    time.sleep(0.1)
    #requests.get(entry)
    pass_soup = BeautifulSoup(requests.get(df['links'][i]).content, "html.parser")
    pass_image_links = pass_soup.find_all(find_by_data_test_id_custom)
    
    pass_image_src = [ link.get('src') if link.has_attr('src') 
       else link.get('data-flickity-lazyload') 
       for link in img_links ]

    pass_image_alt = [ link.get('alt') for link in pass_image_links]
    
    if len(pass_image_links) > 1: 
        img_list[df['id_int'][i]] =  {'src':pass_image_src} 
        
img_list


{'769424965': {'src': ['https://cache.willhaben.at/mmo/1/769/424/561_1110958152.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_1794694154.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_1880484276.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_960294679.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_1295309011.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-356202943.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-84416267.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-681580211.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-1413034130.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-424420074.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-775431706.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-245896278.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-273355802.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-1358047413.jpg',
   'https://cache.willhaben.at/mmo/1/769/424/561_-160773

In [83]:
df_src = pd.DataFrame(img_list).T.explode('src').reset_index(names='id')

In [ ]:
# GENERATE A COLUMN WITH A PREFIX

# Download all the pictures
Now, it is essential to find a way to rename the figures like: "stolen_78999345" or "not_stolen_5643345"

In [99]:
import io
import requests

# Create a byte object out of image_content and store it in the variable image_file
image_file = io.BytesIO(requests.get(df_src['src'][0]).content)

from PIL import Image
image = Image.open(image_file).convert("RGB")

# pathlib lets you point to specific directories.
from pathlib import Path
print(Path().absolute())

# assuming there is a folder "figs" in the current working directory
file_path = Path("figs","stolen_" + df_src['id'][0] + ".png").absolute()
print(file_path)
image.save(file_path, "PNG", quality=80)


C:\Users\georg\Documents\GIT\fastbook\bike-reloot
C:\Users\georg\Documents\GIT\fastbook\bike-reloot\figs\stolen_769424965.png


In [ ]:
# Now iterate through the list 